## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.insert(0, "../")

In [ ]:
import templatefitter as tf

## Set up samples

In [ ]:
mu_sig_mc = 1/4
mu_bkg_mc = 1 - mu_sig_mc
mu_sig_data = 1/3
mu_bkg_data = 1 - mu_sig_data

In [ ]:
n_mc = 1000
n_data = 1000

In [ ]:
kinvar = "kinvar"
kinvar_limits = (0, 10)

In [ ]:
def gen_sig(size):
    return np.random.normal((kinvar_limits[1]+kinvar_limits[0])/2, 1, int(size))

def gen_bkg(size):
    return kinvar_limits[0] + (kinvar_limits[1]-kinvar_limits[0])*np.random.random_sample(int(size))

In [ ]:
# Gaussian around 5
mc_sig = pd.DataFrame()
mc_sig[kinvar] = gen_sig(mu_sig_mc * n_mc)
mc_sig["issig"] = False

In [ ]:
# Uniform distribution 0 to 10
mc_bkg = pd.DataFrame()
mc_bkg[kinvar] = gen_bkg(mu_bkg_mc * n_mc)
mc_bkg["issig"] = True

In [ ]:
mc = pd.concat([mc_sig, mc_bkg])

In [ ]:
# Data distribution:
data = pd.DataFrame()
data[kinvar] = np.concatenate((gen_sig(mu_sig_data * n_data), gen_bkg(mu_bkg_data * n_data)))

In [ ]:
# mc.hist(kinvar, by="issig")
# mc.hist(kinvar)
# data.hist(kinvar)

## Set up templates

In [ ]:
num_bins = 5

In [ ]:
h_sig = tf.histograms.Hist1d(bins=num_bins, range=kinvar_limits, data=mc_sig[kinvar])
t_sig = tf.templates.Template1d("Signal", kinvar, h_sig, color="cornflowerblue")
h_bkg = tf.histograms.Hist1d(bins=num_bins, range=kinvar_limits, data=mc_bkg[kinvar])
t_bkg = tf.templates.Template1d("Background", kinvar, h_bkg, color="indianred")

In [ ]:
for template in [t_sig, t_bkg]:
    fig, axis = plt.subplots(1,1)
    template.plot_on(axis)

In [ ]:
mct = tf.templates.MultiChannelTemplate()

In [ ]:
mct.define_channel("default", num_bins, kinvar_limits)

In [ ]:
mct.define_process("sig")
mct.define_process("bkg")

In [ ]:
mct.add_template("default", "sig", t_sig)
mct.add_template("default", "bkg", t_bkg)

In [ ]:
mct.add_data(default=tf.histograms.Hist1d(num_bins, kinvar_limits, data=data[kinvar]))

In [ ]:
fig, axis = plt.subplots(1,1, figsize=(8,8))
mct.channels["default"].plot_stacked_on(axis)

## Fit

In [ ]:
fitter = tf.TemplateFitter(mct, "iminuit")

In [ ]:
fitter.do_fit(update_templates=True)

In [ ]:
fig, axis = plt.subplots(1,1, figsize=(8,8))
mct.channels["default"].plot_stacked_on(axis)